#<font color="Green">Vocal Remover 5.0.2/c1.04</font>
<br>Trained models provided in this notebook are by [Anjok07](https://github.com/Anjok07), and [aufr33](https://github.com/aufr33).</br>
<br>OFFICIAL UVR GITHUB PAGE: [here](https://github.com/Anjok07/ultimatevocalremovergui).</br>
<br>OFFICIAL CLI Version: [here](https://github.com/tsurumeso/vocal-remover).</br>
<br>Join our [Discord server](https://cutt.ly/0TcDjmo)!</br>
<sup><br>Powered by [tsurumeso](http://github.com/tsurumeso/).
<br>Colaboratory version by [AudioHacker](https://www.youtube.com/channel/UC0NiSV1jLMH-9E09wiDVFYw).</br></sup>

In [ ]:
# ------------VSC REWRITE
# pyright: reportMissingImports=false, reportUnusedVariable=warning, reportUntypedBaseClass=error
#@title Mount to Drive

#update channels
ai = 'https://github.com/NaJeongMo/Colaboratory-Notebook-for-Ultimate-Vocal-Remover'

vercheck = 'https://raw.githubusercontent.com/NaJeongMo/Colaboratory-Notebook-for-Ultimate-Vocal-Remover/main/v'

model_ver = 'https://raw.githubusercontent.com/NaJeongMo/Colaboratory-Notebook-for-Ultimate-Vocal-Remover/main/model_list'

from IPython.display import Audio, display
from google.colab import output
from google.colab import drive
from google.colab import files
from sys import exit
import zipfile
import hashlib
import os.path
import shutil
import psutil
import random
import glob
import time
import zlib
import sys
import os
#@markdown Uncheck if you want to use VocalRemover5 without mounting to drive.
MountDrive = True #@param{type:"boolean"}
#@markdown Mounting path; don't touch this if you don't know what you're doing
mounting_path = '/content/drive/MyDrive' #@param ["snippets:","/content/drive/MyDrive","/content/drive/Shareddrives/<your shared drive name>", "/content/drive/Shareddrives/Shared Drive"]{allow-input: true}
#@markdown Force trigger update
ForceUpdate = False #@param{type:"boolean"}
## @markdown Reinstall VR
# flush = False #@param{type:"boolean"}
## output.clear()
class hide_opt: # hide outputs
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def get_size(bytes, suffix='B'): # read ram
    global svmem
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f'{bytes:.2f}{unit}{suffix}'
        bytes /= factor
    svmem = psutil.virtual_memory()

def validateModelLinks():
    with hide_opt():
        get_ipython().system('wget {} -O model_ver'.format(model_ver))
    model_ver_ = open("model_ver", "r")
    model_ver_ = model_ver_.read()
    with hide_opt():
        get_ipython().system('wget {} -O model_list'.format(model_ver_))
    model_list = open("model_list", "r")
    model_list = model_list.readlines()
    models = []
    for i in model_list:
        models.append(i)
    os.remove('model_ver')
    os.remove('model_list')
    return models

def installAI():
    print('Installing ai...', end=' ')
    with hide_opt():
        get_ipython().system('git clone {} VocalRemover5-COLAB_arch'.format(ai))
        os.chdir('VocalRemover5-COLAB_arch')
    print('done')

    print('Downloading models...', end=' ')
    for i in validateModelLinks():
        with hide_opt():
            zname = sanitize_filename(os.path.basename(i))
            get_ipython().system("wget {}".format(i))
            get_ipython().system('unzip -o {}'.format(zname))
            os.remove(zname)
    print('done')

def mount_to_colab(): # cd to drive
    os.chdir(f'{mounting_path}/VocalRemover5-COLAB_arch')
    print('Success!')

def vrmount(): # install ai and or cd to drive
    drive.mount("/content/drive", force_remount=True)
    os.chdir(mounting_path + '/')
    if os.path.isdir(f'{mounting_path}/VocalRemover5-COLAB_arch'):
        check_update()
        mount_to_colab()
    else:
        print("It looks like it's your first time using VocalRemover5!")
        print('About 4.5 gb of Google Drive will be used. This will take around 2-3 mins.')
        print('Make sure to clear your trash bin!!')
        confirmation = input('Do you want to continue? (y/n): ')
        if confirmation.lower() == 'y':
            print('')
            installAI()
            mount_to_colab()
        else:
            print('Please uncheck MountDrive')
            exit()

def dlvr(): # download vr to colab only
    print('Warning: changes will not be saved.')
    print('Downloading VR5...')
    with hide_opt():
        os.chdir('/content')
    installAI()
    #os.chdir('/content/VocalRemover5-COLAB_arch')
    print ("Success!")

def check_update():
    if os.path.isdir(f'{mounting_path}/VocalRemover5-COLAB_arch'): # check update if ai installed is True
        os.chdir(f'{mounting_path}/VocalRemover5-COLAB_arch')
        print("Checking for updates...", end=" ")
        with hide_opt():
            get_ipython().system('wget {} -O check_ver'.format(vercheck))
        f = open("check_ver", "r")
        nver = f.read()
        f = open("v", "r")
        cver = f.read()
        if cver != nver or ForceUpdate:
            print('New update found! {}'.format(nver))
            os.chdir('../')
            print('Updating ai...',end=' ')
            with hide_opt():
                get_ipython().system('git clone {} temp_VocalRemover5-COLAB_arch'.format(ai))
            get_ipython().system('cp -a temp_VocalRemover5-COLAB_arch/* VocalRemover5-COLAB_arch/')
            get_ipython().system('rm -rf temp_VocalRemover5-COLAB_arch')
            print('done')

            print('Downloading models...', end=' ')
            os.chdir('VocalRemover5-COLAB_arch')
            for i in validateModelLinks():
                with hide_opt():
                    zname = sanitize_filename(os.path.basename(i))
                    get_ipython().system("wget {}".format(i))
                    get_ipython().system('unzip -o {}'.format(zname))
                    os.remove(zname)
            print('done')
            output.clear()
            os.remove(f'{mounting_path}/VocalRemover5-COLAB_arch/v')
            os.rename(f'{mounting_path}/VocalRemover5-COLAB_arch/check_ver',f'{mounting_path}/VocalRemover5-COLAB_arch/v')
            os.chdir(f'{mounting_path}/VocalRemover5-COLAB_arch') # just to make sure
        else:
            os.remove('check_ver')
            print('No update found.')
    else:
        if MountDrive:
            vrmount() # colab
        elif MountDrive == False:
            if os.path.isdir('/content/VocalRemover5-COLAB_arch'):
                print("Success!")
            else:
                dlvr()
# Import other modules for cell-inference
with hide_opt():
    ! pip install pathvalidate
    ! pip install youtube-dl
    ! pip install samplerate

    #! pip install librosa==0.8.0
    #! pip install numba==0.48.0
    #! pip install resampy==0.2.2

    from pathvalidate import sanitize_filename # for inference cell
    import youtube_dl # for inference cell
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
isCpu = False
if gpu_info.find('failed') >= 0:
    svmem = psutil.virtual_memory()
    print(f'GPU runtime is disabled. You have {get_size(svmem.total)} RAM.')
    isCpu = True
if gpu_info.find('Tesla T4') >= 0:
    print('You got a Tesla T4 GPU. (speeds are around  10-25 it/s)')
elif gpu_info.find('Tesla P4') >= 0:
    print('You got a Tesla P4 GPU. (speeds are around  8-22 it/s)')
elif gpu_info.find('Tesla K80') >= 0:
    print('You got a Tesla K80 GPU. (This is the common gpu, speeds are around 2-10 it/s)')
elif gpu_info.find('Tesla P100') >= 0:
    print('You got a Tesla P100 GPU. (This is the FASTEST gpu, speeds are around  15-42 it/s)')
else:
    if isCpu == False:
        print('You got an unknown GPU. Please report the GPU you got!')
        ! nvidia-smi
#----------DEFINE INFERENCE REQ------------
def crc32(fileName):
    with open(fileName, 'rb') as fh:
        hash = 0
        while True:
            s = fh.read(65536)
            if not s:
                break
            hash = zlib.crc32(s, hash)
        return "%08X" % (hash & 0xFFFFFFFF)
def YouTube(link, dl=True):
    inputsha = hashlib.sha1(bytes(link, encoding='utf8')).hexdigest() + '.wav'
    fmt = '251/140/250/139'
    opt = {'format': fmt, 'outtmpl': inputsha, 'updatetime': False, 'nocheckcertificate': True}
    if dl == True:
        print('YouTube link detected')
        print('Downloading...', end=' ')
    with hide_opt():
        with youtube_dl.YoutubeDL(opt) as ydl:
            global desc
            if dl == True:
                desc = ydl.extract_info(link, download=not os.path.isfile(inputsha))
            else:
                desc = ydl.extract_info(link, download=not True)

    titlename = sanitize_filename(desc['title'])
    if dl == True:
        print('done')
        print(titlename)
    if dl == True:
        return titlename, inputsha
    else:
        return titlename
def zipdir(folder, zipname): # LINUX CALL MODIFIED!!!
    if '.zip' in zipname:
        zipname = zipname.strip('.zip')
    get_ipython().system('zip -r {}.zip {}'.format(zipname,folder))
# dlFile(input,pretrained_model,isYouTube='http://' in input,export_as_mp3=export_as_mp3)
def dlFile(track,pretrained_model,isYouTube=False,export_as_mp3=False):
    modelname = os.path.splitext(os.path.basename(pretrained_model))[0]
    stems = [f'_{modelname}_Instruments',f'_{modelname}_Vocals']
    filename = os.path.splitext(os.path.basename(track))[0]
    if isYouTube:
        filename = YouTube(track,dl=False)
    if os.path.isdir(filename):
        shutil.rmtree(filename)
        os.mkdir(filename)
    else:
        os.mkdir(filename)
    if os.path.isfile(f'{filename}.zip'):
        os.remove(f'{filename}.zip')
    if export_as_mp3:
        os.chdir('separated/')
        for i in stems:
            wav_path = filename + i + '.wav'
            mp3_path = filename + i + '.mp3'
            get_ipython().system(f'ffmpeg -y -i "{wav_path}" -vn -ar 44100 -ac 2 -b:a 320k "{mp3_path}" -loglevel quiet')
        os.chdir('../')
        for move in stems:
            shutil.move('separated/' + filename + move + '.mp3',filename)
    else:
        for move in stems:
            shutil.move('separated/' + filename + move + '.wav',filename)
    with hide_opt():
        get_ipython().system(f'zip -r "{filename}.zip" "{filename}"')
    shutil.rmtree(filename)
    files.download(f'{filename}.zip')

#-------------Script begin-------------

if MountDrive:
    vrmount()
else:
    if os.path.isdir('/content/VocalRemover5-COLAB_arch') == False:
        if os.path.isdir('/content/VocalRemover5-COLAB_arch'):
            print('Success!')
        else:
            dlvr()


#<font color="Blue">Print a list of tracks and models</font>

In [ ]:
if os.path.isfile('main.py') == False:
  if MountDrive:
    os.chdir(mounting_path + '/VocalRemover5-COLAB_arch')
  else:
    os.chdir('/content/VocalRemover5-COLAB_arch')
#@title <- Press me
#@markdown Clear tracks folder
clearTracks = False #@param {type:"boolean"}
#@markdown Clear separated folder
clearSeparated = False #@param {type:"boolean"}
print("Here's a list of files in your tracks folder:")
print("")
if clearTracks:
    for i in glob.glob('tracks/*'):
        os.remove(i)
! ls tracks/
print("")
print("---------------------------------------------------")
print("")
print("Usable models:")
print("")
print("   v5 (new arch):")
! ls models/v5_new
print("   v5:")
! ls models/v5
print("---------------------------------------------------")
print("")
print("Usable parameters:")
print("")
! ls modelparams/
print("---------------------------------------------------")
print("")
print("Your separated files:")
print("")
if clearSeparated:
    for i in glob.glob('separated/*'):
        os.remove(i)
! ls separated/

#Inference

In [ ]:
# ------------VSC REWRITE
#type: ignore
if os.path.isfile('main.py') == False:
    if MountDrive:
        os.chdir(mounting_path + '/VocalRemover5-COLAB_arch')
    else:
        os.chdir('/content/VocalRemover5-COLAB_arch')

#parameter markdowns-----------------

#@markdown ### Input files
#@markdown Track name / any link (Upload your songs in tracks folder)
input = "Enter any link or filename" #@param {type:"string"}
ScanSeparatedFolder = False #@param {type:"boolean"}
#@markdown Convert all files in your tracks folder
convertAll = False #@param {type:"boolean"}
if convertAll:
    convertAll = '--convert_all'
else:
    convertAll = ''
#@markdown Model name (Upload your models in models folder)
pretrained_model = "HighPrecison_4band_arch-124m_1.pth" #@param ["HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
#@markdown ---
#@markdown ### Arguments
window_size =  512#@param {type:"integer"}
if window_size < 320:
    print('Warning: window_size lower than 320.')
if window_size < 272:
    window_size = 272
#1band_sr32000_hl512.json  2band_44100_lofi.json  3band_44100_mid.json
#1band_sr44100_hl512.json  2band_48000.json	 4band_44100.json
#2band_32000.json	  3band_44100.json	 ensemble.json
parameter = "Auto detect" #@param ["Auto detect","1band_sr32000_hl512.json","1band_sr44100_hl512.json", "2band_32000.json" , "2band_44100_lofi.json", "2band_48000.json", "3band_44100.json", "3band_44100_mid.json", "3band_44100_msb2.json", "4band_44100.json"]
parameter = "modelparams/" + parameter
high_end_process = 'mirroring2' #@param ["none","mirroring", "mirroring2" , "bypass"]
aggressiveness = '0.3' #@param {type:"string"}
aggressiveness = float(aggressiveness)
#@markdown Mute low volume vocals
postprocess = False #@param {type: "boolean"}
if postprocess:
    threshold = 0.2 #?param {type:"number"}
    min_range = 64 #?param {type:"integer"}
    fade_size = 32 #?param {type:"integer"}
    if min_range < fade_size * 2:
        print('min_range must be greater than or equal fade_size * 2')
        print('Using default instead. (except threshold)')
        min_range = 32
        fade_size = 64
    postprocess = f"-p -thres {threshold} -mrange {min_range} -fsize {fade_size}"
else:
    postprocess = ""
#@markdown ### Architecture
nn_architecture = 'Auto detect' #@param ["Auto detect","default", "34 MB", "124 MB", "130 MB", "500 MB"]
if nn_architecture == '34 MB':
    nn_architecture = '33966KB'
elif nn_architecture == '124 MB':
    nn_architecture = '123821KB'
elif nn_architecture == '130 MB':
    nn_architecture = '129605KB'
elif nn_architecture == '500 MB':
    nn_architecture = '537238KB'
elif nn_architecture == 'default':
    pass
#@markdown ---
#@markdown ### Checkboxes
#@markdown Use GPU for faster conversion
gpu = True #@param {type: "boolean"}
if gpu == True:
    gpu = 0
else:
    gpu = -1
#@markdown Aggressively remove vocals from Instrumental
deepExtraction = False #@param {type:"boolean"}
if deepExtraction:
    deepExtraction = "-D"
else:
    deepExtraction = ""
#@markdown Flip Instruments and Vocals output (Only for Vocal Models)
isVocal = False #@param {type:"boolean"}
if isVocal:
    isVocal = '--isVocal'
else:
    isVocal = ''
#@markdown Hide warnings
suppress = True #@param {type:"boolean"}
if suppress:
    suppress = '--suppress'
else:
    suppress = ''
#@markdown Export spectogram image
output_image = False #@param {type: "boolean"}
if output_image:
    output_image = "-I"
else:
    output_image = ""
#@markdown perform Test Time Augmentation to improve the separation quality
tta = False #@param {type: "boolean"}
if tta:
    tta = "-t"
else:
    tta = ""
#@markdown Use custom arguments
useCustomArguments = False #@param {type: "boolean"}
CustomArguments = "-h" #@param {type:"string"}
#@markdown Download files
download = False #@param {type:"boolean"}
export_as_mp3 = True #@param {type:"boolean"}
#@markdown Use all model
model_version = "Don't use all model" #@param ["Don't use all model","v5_new", "v5", "all"]
# {none,v5,v5_new,all}
# automation
if nn_architecture == 'Auto detect':
    if 'arch-default' in pretrained_model:
        nn_architecture = 'default'
    elif 'arch-34m' in pretrained_model:
        nn_architecture = '33966KB'
    elif 'arch-124m' in pretrained_model:
        nn_architecture = '123821KB'
    elif 'arch-130m' in pretrained_model:
        nn_architecture = '129605KB'
    elif 'arch-500m' in pretrained_model:
        nn_architecture = '537238KB'
    else:
        print('Error! autoDetect_arch')
        print('Using 124 MB instead.')
        nn_architecture = '123821KB'
if parameter == 'modelparams/Auto detect':
    if '4band' in pretrained_model.lower():
        parameter = 'modelparams/4band_44100.json'
    elif '3band' in pretrained_model.lower():
        if 'msb2' in pretrained_model.lower():
            parameter = 'modelparams/3band_44100_msb2.json'
        else:
            parameter = 'modelparams/3band_44100.json'
    elif 'midside' in pretrained_model.lower():
        parameter = 'modelparams/3band_44100_mid.json'
    elif '2band' in pretrained_model.lower():
        if 'lofi' in pretrained_model.lower():
            parameter = 'modelparams/2band_44100_lofi.json'
        else:
            parameter = 'modelparams/2band_48000.json'
    else:
        print('Parameter auto detect failed, using 1band instead.')
        parameter = 'modelparams/1band_sr44100_hl512.json'
if '34m' in pretrained_model or '124m' in pretrained_model or '130m' in pretrained_model or '500m' in pretrained_model:
    pretrained_model = "models/v5_new/" + pretrained_model
else:
    pretrained_model = "models/v5/" + pretrained_model
    if os.path.isfile(pretrained_model) == False:
        print('========================================================')
        print('                Error model not found.')
        print('Custom models should be uploaded in "models/v5/" folder!')
        print('========================================================')
if 'https://' not in input:
    if ScanSeparatedFolder:
        if input in ''.join(glob.glob('separated/*')):
            input = 'separated/' + input
        else:
            print('File not found in separated folder.')
            input = 'tracks/' + input
    else:
        input = 'tracks/' + input
elif 'https://' in input and ScanSeparatedFolder:
    print('Skipping "Separated" folder scan since a link is given.')
   
if model_version == "Don't use all model":
    model_version = 'none'

# --------------AI----------------
start_time = time.time()
#window_size,os.path.splitext(os.path.basename(pretrained_model)[0]),os.path.splitext(os.path.basename(parameter)[0],settings_tta = 'True' if tta else 'False'
#settings_tta = 'True' if tta else 'False',os.path.splitext(os.path.basename(parameter)[0],os.path.splitext(os.path.basename(pretrained_model)[0]),window_size
settings_tta = 'True' if tta else 'False'
settings_deepExtraction = 'True' if deepExtraction else 'False'

print('Window size: {}'.format(window_size))
print('Model: {}'.format(os.path.splitext(os.path.basename(pretrained_model))[0]))
print('Parameter: {}'.format(os.path.splitext(os.path.basename(parameter))[0]))
print('Aggressiveness: {}'.format(aggressiveness))
print('High end process: {}'.format(high_end_process))

print('TTA: {}'.format(settings_tta))
print('Deep Extraction: {}'.format(settings_deepExtraction))
print()
if useCustomArguments == False:
    get_ipython().system(f'python3 main.py -i "{input}" {convertAll} --useAllModel "{model_version}" --model_params "{parameter}" -P "{pretrained_model}" -w {window_size} -H "{high_end_process}" --aggressiveness {aggressiveness} -n "{nn_architecture}" -g {gpu} {deepExtraction} {isVocal} {suppress} {output_image} {postprocess} {tta}')
    if download and convertAll:
        sys.exit("No no, this is not an error but downloading with convertAll is not yet possible. Please DON'T report this to me (Hv) or the server")
    if download:
        dlFile(input,pretrained_model,isYouTube='https://' in input,export_as_mp3=export_as_mp3)
if useCustomArguments:
    get_ipython().system(f'python3 main.py {CustomArguments}')
print('Notebook took: {0:.{1}f}s'.format(time.time() - start_time, 1))

---


In [ ]:
#@markdown ### Custom Model Ensemble
#@markdown Input tracks
input = "Enter any link or filename" #@param {type:"string"}
#@markdown Convert all files in your tracks folder
ensembling_parameter = "1band_sr44100_hl512.json" #@param ["1band_sr32000_hl512.json","1band_sr44100_hl512.json", "2band_32000.json" , "2band_44100_lofi.json", "2band_48000.json", "3band_44100.json", "4band_44100.json"]
ensembling_parameter = "modelparams/" + ensembling_parameter
convertAll = False #@param {type:"boolean"}
window_size =  1024#@param {type:"integer"}
if window_size < 320:
  print('Warning: window_size lower than 320.')
if window_size < 272:
  window_size = 272
high_end_process = 'none' #@param ["none","mirroring", "mirroring2" , "bypass (Recommended for ensembling!)"]
if high_end_process == 'bypass (Recommended for ensembling!)':
    high_end_process = 'bypass'
aggressiveness = '0.1' #@param {type:"string"}
aggressiveness = float(aggressiveness)
FlipVocalModels = True #@param {type:"boolean"}
if FlipVocalModels:
    FlipVocalModels = '--flipVocals'
else:
    FlipVocalModels = ''
tta = False #@param {type:"boolean"}
if tta:
    tta = '--tta'
else:
    tta = ''
postprocess = False #@param{type:"boolean"}
if postprocess:
    postprocess = '--postprocess'
else:
    postprocess = ''
#@markdown Algorithm
instrumental = 'min_mag' #@param ["min_mag","max_mag","comb_norm","invert"]
vocal = 'max_mag' #@param ["min_mag","max_mag","comb_norm","invert"]
#@markdown Model cocktail ( ͡° ͜ʖ ͡°)
# just a quick note, you can add more model markdowns if you want.
def ensYouTube(link, dl=True):
    inputsha = hashlib.sha1(bytes(link, encoding='utf8')).hexdigest() + '.wav'
    fmt = '251/140/250/139'
    if 'youtu' not in link:
        fmt = 'best'
        print('Non-YouTube link detected. Attempting to download.')
    else:
        print('YouTube link detected')
    opt = {'format': fmt, 'outtmpl': inputsha, 'updatetime': False, 'nocheckcertificate': True}
    if dl == True:
        print('Downloading...', end=' ')
    with hide_opt():
        with youtube_dl.YoutubeDL(opt) as ydl:
            global desc
            if dl == True:
                desc = ydl.extract_info(link, download=not os.path.isfile(inputsha))
            else:
                desc = ydl.extract_info(link, download=False)
    if dl == True:
        print('done')
    titlename = sanitize_filename(desc['title'])
    if dl == True:
        print(titlename)
    if dl==False:
        return titlename
    else:
        return titlename, inputsha

def modelPath(model):
    if model == 'slot':
        model = ''
    if '34m' in model or '124m' in model or '130m' in model or '500m' in model:
        model = "models/v5_new/" + model
    elif model == '':
        pass
    else:
        model = "models/v5/" + model
    return model
def quot():
    return '"'
model_1 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_2 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_3 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_4 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_5 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_6 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_7 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_8 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_9 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_10 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_11 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_12 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_13 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_14 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_15 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_16 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_17 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_18 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_19 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
model_20 = "slot" #@param ["slot","HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}

model_1 = modelPath(model_1)
model_2 = modelPath(model_2)
model_3 = modelPath(model_3)
model_4 = modelPath(model_4)
model_5 = modelPath(model_5)
model_6 = modelPath(model_6)
model_7 = modelPath(model_7)
model_8 = modelPath(model_8)
model_9 = modelPath(model_9)
model_10 = modelPath(model_10)
model_11 = modelPath(model_11)
model_12 = modelPath(model_12)
model_13 = modelPath(model_13)
model_14 = modelPath(model_14)
model_15 = modelPath(model_15)
model_16 = modelPath(model_16)
model_17 = modelPath(model_17)
model_18 = modelPath(model_18)
model_19 = modelPath(model_19)
model_20 = modelPath(model_20)
if 'https://' in input:
    titlename,inputsha=ensYouTube(input)
    get_ipython().system(f"python ensemble_colab.py --ens_param {ensembling_parameter} --algo {instrumental} {vocal} {postprocess} {tta} {FlipVocalModels} --aggressiveness {aggressiveness} -w {window_size} --high_end_process {high_end_process} --suppress --input '{inputsha}' --model_ens {model_1} {model_2} {model_3} {model_4} {model_5} {model_6} {model_7} {model_8} {model_9} {model_10} {model_11} {model_12} {model_13} {model_14} {model_15} {model_16} {model_17} {model_18} {model_19} {model_20}")
    os.rename(rf'separated/{os.path.splitext(os.path.basename(inputsha))[0]}_Ensembled_Instruments.wav',rf'separated/{titlename}_Ensembled_Instruments.wav')
    os.rename(rf'separated/{os.path.splitext(os.path.basename(inputsha))[0]}_Ensembled_Vocals.wav',rf'separated/{titlename}_Ensembled_Vocals.wav')
else:
    if convertAll:
        for track in glob.glob('tracks/*'):
            get_ipython().system(f"python ensemble_colab.py --ens_param {ensembling_parameter} --algo {instrumental} {vocal} {postprocess} {tta} {FlipVocalModels} --aggressiveness {aggressiveness} -w {window_size} --high_end_process {high_end_process} --suppress --input '{track}' --model_ens {model_1} {model_2} {model_3} {model_4} {model_5} {model_6} {model_7} {model_8} {model_9} {model_10} {model_11} {model_12} {model_13} {model_14} {model_15} {model_16} {model_17} {model_18} {model_19} {model_20}")
    else:
        input = 'tracks/' + input
        get_ipython().system(f"python ensemble_colab.py --ens_param {ensembling_parameter} --algo {instrumental} {vocal} {postprocess} {tta} {FlipVocalModels} --aggressiveness {aggressiveness} -w {window_size} --high_end_process {high_end_process} --suppress --input '{input}' --model_ens {model_1} {model_2} {model_3} {model_4} {model_5} {model_6} {model_7} {model_8} {model_9} {model_10} {model_11} {model_12} {model_13} {model_14} {model_15} {model_16} {model_17} {model_18} {model_19} {model_20}")

report errors to `Hv#3868`
--

###**Best settings in my opinon:**
<h3><b>Instrumental:</b></h3>
<p><sup>Window size: 320</sup></p>
<p><sup>Model: HP2-4BAND-3090_4band_arch-500m_1.pth</sup></p>
<p><sup>Parameter: 4band_44100.json</sup></p>
<p><sup>TTA: ON</sup></p>
<p><sup>Aggressiveness: 0.3</sup></p>